# 極性判定とDoc２Vecを使ったTwitterネガポジ予測
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
### 【このnotebookについて】
2019年7〜10月までフルタイムで通っていたスクールの卒業課題テーマを、機械学習の勉強のために発展させたものです<br>
卒業発表スライド　https://www.slideshare.net/secret/y0m7g1nZdxpVYP<br>
＊当初は炎上予測がテーマだったので、このnotebookの内容とはややズレます<br>
＊表紙スライドの字が見えない場合は２枚目から戻ると見えます<br>

＊ちなみに…<br>
スクールで取り組んだ課題のリポジトリ 
https://github.com/kaorisugi/diveintocode-ml<br>
論文読解課題のスライドシェア 
https://www.slideshare.net/secret/qGmdiwl4uGS20O<br>

### 【ゴール】
これからツイートする予定の文章に対し、過去の類似ツイートを探し、反応のネガポジスコア付きで上位１０位まで提示する。<br>
### 【モデルの仕組み】
１）ツイートデータセットを取得<br>
　・TwitterAPIを使ってツイートを取得<br>
　・各ツイートに対する反応ツイート（リプライ、引用RT）を取得<br>
　・反応ツイートの極性表現数をカウントしてネガポジスコアとpositive/negative/fire!!!判定を得る<br>
 　（positive/negativeの判定基準：極性表現数が多い方、fire!!!(炎上）の判定基準：極性表現の７０％以上がnegative）<br>
２）データセットの前処理<br>
　・正規表現、ストップワード除去など<br>
３）予測モデルを生成<br>
　・データセットをDoc２vecで学習<br>
４）ツイート予定文章のネガポジ予測を返す<br>
　・データセットから、ツイート予定文書と似ている文書を探す<br>
 ・ネガポジスコア付きで、類似ツイート上位１０個を返す
### 【結果】
類似度確認用にデータセット内にあるものと同じ文を入力したところ、類似度1位で返ってきた。また、２位、３位にもマスクに関する似た話題のツイートが提示されたので類似ツイートの抽出は成功。ネガポジスコアもデータズレなどなく正確に表示され、目的は達成できた。<br>
ツイッターAPI制限により、まだサンプルが少ない（完成時２００件程度）が、データを蓄積できる仕様にしているので、ツイート文のバリエーションを増やしていけば、様々な入力文に対応できるようになると思う。<br>
ネガポジ判定については、ネガティブなテーマへの言及に共感したコメントでネガ判定が出ているケースも多く、必ずしもツイート主へのネガ感情ではないことに注意が必要。<br>

### 【その他試みたこと】
１）文章ベクトルを特徴量としたネガポジ予測モデル<br>
　・文章ベクトルとフォロワー数を特徴量X、ネガポジスコアを目的変数yとしたデータを学習<br>
　・文章ベクトルはDoc２vecとTf-idfの２種を作成<br>
　・ツイート予定文書を入力してネガポジスコアを予測する<br>
　・試した予測モデル<br>
　・MultiOutputRegressor、SVRのrbf と　SVRの線形、lightgbm、ランダムフォレスト<br>
　  　→精度が低すぎて断念<br>
２）ツイッターAPI制限への挑戦（データセットの拡大）<br>
　・古いツイートを大量取得できるパッケージを発見（通常は１週間程度しか遡れない）<br>
　　　→取得データから反応ツイートの取得を試みたができなかった<br>
   
### 【利用するには】
・config.py ファイルにツイッターAPIトークンを記入<br>
・インストールが必要なツールは、notebook内にマジックコマンドにて記載してあります<br>
・jupyternotebook上でのMeCabの利用で詰まる場合は、下記のDockerイメージを使うとうまくいくかと思います。<br>
Docker Japanese NLP<br>
https://github.com/hoto17296/docker-japanese-nlp<br>
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝

## １）ツイートデータセットを取得
・TwitterAPIでツイートを取得<br>
・各ツイートに対するリプライ、引用RTを取得<br>
・極性表現数をカウントしてネガポジスコアを得る<br>

#### 参考サイト
【Python】tweepyでTwitterのツイートを検索して取得<br>
https://vatchlog.com/tweepy-search/<br>
【Python】tweepyで期間指定してツイートを検索する<br>
https://vatchlog.com/tweepy-search-time/<br>
バズったツイートへのリアクションを感情分析してみる<br>【Google Natural Language API / Python】<br>
https://qiita.com/matsuri0828/items/029b4d0d510dcfb5c5dd

In [3]:
#必要なツールをインストール（初回のみ実行）
! pip install --upgrade pip
! pip install tweepy
! pip install oseti
! pip install requests requests_oauthlib
! pip install sengiri

    100% |████████████████████████████████| 1.4MB 713kB/s 
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1
     |████████████████████████████████| 153kB 11.9MB/s 
     |████████████████████████████████| 81kB 4.4MB/s 
     |████████████████████████████████| 51kB 11.8MB/s 
  Created wheel for oseti: filename=oseti-0.2-cp36-none-any.whl size=75997 sha256=fa59bef52cdf3ac5af8d719db2eb3a2d4039c64d698cf2ac99807caeba04a01d
  Stored in directory: /root/.cache/pip/wheels/3f/a0/b8/306332f20803a7af8bbb35ab3ddaf625dfef62076a6372eec7
  Created wheel for sengiri: filename=sengiri-0.2.1-cp36-none-any.whl size=5545 sha256=fb4986a7e7eb7710f27046c0a02362d815672d79ba9e14742bdee52327630f24
  Stored in directory: /root/.cache/pip/wheels/dd/c1/36/d123492db6ec7445fe4124351505e1150a35e03e5c08f3dad0
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=43030 sha256=9c5155f8758d5d3ee35d557362c3e5a15d2fa9927ff2082ef4a3c1f57da28fcb
  St

In [1]:
import tweepy
import re
import emoji
import oseti
from datetime import datetime, date, timedelta, time
import time
import os
import pandas as pd
import csv
from tqdm import tqdm
import config

class Get_Twitter():

    def __init__(self, day, reload, print_rep = False, exclud_words = "配信スタート ＃キャンペーン　リツイートキャンペーン", RT_count = 5000):
        self.oseti_analyzer = oseti.Analyzer()  #極性判定
        self.CK = config.CONSUMER_KEY
        self.CS = config.CONSUMER_SECRET
        self.AT = config.ACCESS_TOKEN
        self.AS = config.ACCESS_TOKEN_SECRET
        self.ew = exclud_words
        self.print_rep = print_rep
        self.rt = str(RT_count)
        self.columns = [
            "Id", "Date", "Name", "Full_text",
            "Judge", "Posi_score", "Nega_score", "Followers", "link"
        ]
        self.posi_pd = pd.DataFrame([], columns = self.columns)
        self.nega_pd = pd.DataFrame([], columns = self.columns)
        self.fire_pd = pd.DataFrame([], columns = self.columns)
        self.wait = 0
        self.reload = reload
        day = datetime.strptime(day, '%Y-%m-%d')
        self.day = day.strftime('%Y-%m-%d')

    def main(self):
        self._Make_Dir() # データ格納ファイルの準備

        #ツイートを取得、センチメント判定
        try:
            status = self.Get_Buzz() #バズったツイート取得
            for i in status:            
                if self.wait == 10:
                    print("10回待機したため終了")
                    break
                self.Status(i)
                if self.Exclude_Word(self.buzz_full_text) == True:# 除外ワードを含むツイートは除外
                    continue
                if self.Text_Count() == True: #30W以下のツイートは除外
                    continue
                self.Get_Rep() #リプライを取得
                self.Get_RT() #RTコメントを取得
                if self.Min_Rep() == False: # コメントが少ないツイートは除外
                    continue
                self.Get_Senti() #コメントをセンチメント判定
                self._Get_Analysis() #ツイートをセンチメント判定
        #エラー時はスキップして次のツイート取得
        except (ValueError,  KeyError, TypeError, tweepy.TweepError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except tweepy.RateLimitError as e:
            if self.reload:
                self.wait += 1
                print("==========")
                print('get_buzzのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass
        
        #生成したデータをprint
        print()
        print("↓↓↓positiveサンプル↓↓↓")
        display(self.posi_pd.head())
        print()            
        print("↓↓↓negativeサンプル↓↓↓")
        display(self.nega_pd.head())
        print()
        print("↓↓↓fire_tweetサンプル↓↓↓")
        display(self.fire_pd.head())
        print()
        print()
        
        #生成したPandasDataFrameをcsvで書き出す
        total_pd = pd.concat([self.posi_pd, self.nega_pd, self.fire_pd], ignore_index=True)
        buzz_old = pd.read_csv('./output/buzz_tweet.csv')
        buzz_new = pd.concat([buzz_old, total_pd])#既存データと連結
        buzz_new.drop_duplicates(subset="Id",inplace=True)#重複ID行を削除            
        buzz_new.to_csv('./output/buzz_tweet.csv', index = False, header = True)
        print("csvへの書き出しが完了しました。新規データ数{}、全データ数：{}".format(len(buzz_new) - len(buzz_old), len(buzz_new)))
        print("サンプルが0件の場合は、15分後に再度実行すると取得できる場合があります。") 
        print("fire_tweetは出現率が非常に低いです。")

    #Api認証
    def _Auth(self):
        auth = tweepy.OAuthHandler(self.CK, self.CS)
        auth.set_access_token(self.AT, self.AS)
        api = tweepy.API(auth)
        return api

    #出力用ディレクトリとcsvファイルを作成（存在しない場合のみ）
    def _Make_Dir(self):
        new_dir_path = 'output'
        try:
            os.makedirs(new_dir_path)
        except FileExistsError:
            pass
        if (os.path.isfile('./output/buzz_tweet.csv')) == False:
            self.posi_pd.to_csv('./output/buzz_tweet.csv', index = False)  

    #絵文字削除
    def _remove_emoji(self, text):
        return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)

    #テキストを正規表現処理、絵文字削除
    def _format_text(self, text):
        text=re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
        text=re.sub('\n', "", text)
        text=re.sub(r'@?[!-~]+', "", text)
        text=self._remove_emoji(text)
        return text
    
    #　日付表記を整える、日本時間に修正
    def _date_format(self, date):
        date = datetime.strptime(str(date), '%a %b %d %H:%M:%S %z %Y')
        date = date + timedelta(hours=9)
        return datetime.strftime(date, '%Y-%m-%d %H:%M')

    def Status(self, status): 
        self.buzz_id = status._json['id']
        self.buzz_id_str = status._json['id_str']
        self.buzz_name = status._json['user']['screen_name']
        self.buzz_full_text = status._json['full_text']
        self.date = status._json['created_at']
        self.date = self._date_format(self.date)
        self.favo = status._json['favorite_count']
        self.rt_count = status._json['retweet_count']
        api = self._Auth()
        self.followers = status._json['user']['followers_count']
        #self.followers = len(api.followers(status._json['user']['screen_name']))
    
    #除外ワード
    def Exclude_Word(self, text):                        
        if self.ew in str(text):
            print("==========")
            print("除外ワード")
            print()
            return True
        else:
            return False

    #ツイート内にリンクがあれば分割
    def Text_Count(self):
        if re.search("(https://t.co/\w+)", self.buzz_full_text) == None:
            self.link = None
        else:                   
            self.buzz_full_text = re.split("(https://t.co/\w+)", self.buzz_full_text)
            self.link = self.buzz_full_text[1]
            self.buzz_full_text = self.buzz_full_text[0]
        if len(self.buzz_full_text) < 30:
            return True

    #リプライ＋引用RTコメントが100未満のツイートは除外
    def Min_Rep(self):
        reply_texts_rows = []
        if self.rep_cnt + self.RTcomme_cnt > 100:
            reply_texts_rows.append(self.rep_row)
            reply_texts_rows.append(self.rt_row)
            return True
        else:
            return False

    #sentiment_listを一次元にし、ツイートごとの極性表現の総和の辞書にする
    def Get_Senti(self):
        self.sentiment_list = sum(self.sentiment_list, [])#１次元にする
        self.sentiment = dict((key, sum(d[key] for d in self.sentiment_list)) for key in self.sentiment_list[0])

    #バズったツイートを取得(デフォルト：5000RT以上)
    def Get_Buzz(self):
        api = self._Auth()
        try:
            status = api.search(q = 'filter:safe min_retweets:' + self.rt + ' exclude:retweets until:' + self.day,
                lang ='ja', count =100, tweet_mode = 'extended', result_type = 'recent')
            return status
        #エラー時はスキップして次のツイート取得
        except (ValueError,  KeyError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except (tweepy.RateLimitError, tweepy.TweepError) as e:
            if self.reload:
                self.wait += 1
                print("==========")
                print('get_buzzのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass
        #return status
    
    #リプライを取得
    def Get_Rep(self):
        api = self._Auth()     
        query_reply = '@' + self.buzz_name + ' exclude:retweets'
        self.rep_row = []
        self.sentiment_list = []
        self.rep_cnt =0
        wait_cnt = 0
        try:
            for status_reply in api.search(q=query_reply, lang='ja', count=100):
                if status_reply._json['in_reply_to_status_id_str'] == self.buzz_id_str:
                    row = self._format_text(status_reply._json['text'])
                    #極性判定
                    sentiment_score = self.oseti_analyzer.count_polarity(str(row))#strにする
                    self.sentiment_list.append(sentiment_score)
                    self.rep_row.append(row)
                    self.rep_cnt += 1
                else:
                    pass
        #エラーはスキップして次のツイート取得
        except (ValueError,  KeyError, tweepy.TweepError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except tweepy.RateLimitError as e:
            self.wait += 1
            if self.reload:
                print("==========")
                print('get_repのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass

    # 引用RTを取得
    def Get_RT(self):
        api = self._Auth()
        query_quote = self.buzz_id_str + ' exclude:retweets'
        self.RTcomme_cnt = 0
        self.rt_row = []
        try:
            for status_quote in api.search(q=query_quote, lang='ja', count=100):
                if status_quote._json['id_str'] == self.buzz_id_str:
                    continue
                else:
                    row = self._format_text(status_quote._json['text'])
                #極性判定
                sentiment_score = self.oseti_analyzer.count_polarity(str(row))#strにする
                self.sentiment_list.append(sentiment_score)
                self.rt_row.append(row)
                self.RTcomme_cnt += 1
        #エラーはスキップして次のツイート取得
        except (ValueError,  KeyError, tweepy.TweepError) as e:
            pass
        #リクエスト回数が上限に達した場合はリセット時間まで待機して継続
        except tweepy.RateLimitError as e:
            self.wait += 1
            if self.reload:
                print("==========")
                print('get_rtのリクエスト回数が上限に達しました。リセット時間まで待機')
                print('Wait 15min...')
                print()
                for _ in tqdm(range(15 * 60)):
                    time.sleep(1)
            else:
                pass        

    #取得したTweetをprint
    def _Print(self):
        print("name：", self.buzz_name, "／フォロワー数：", self.followers)
        print("date：", self.date, "／ツイートID：", self.buzz_id_str)
        print("RT数：", self.rt_count, "／favorite数：", self.favo)
        print("リプライ数：", self.rep_cnt, "／RTコメント数(上限１００）：", self.RTcomme_cnt)
        if self.print_rep == True:
            print("リプライ\n", self.rep_row)
            print("RTコメント\n", self.rt_row)
        else:
            pass

    #センチメント判定結果を取得
    def _Get_Analysis(self):
        total = self.sentiment["positive"] + self.sentiment["negative"]
        if self.sentiment["positive"] >= self.sentiment["negative"]:
            print("==========")
            print(self.buzz_full_text)
            print()
            print("【判定:positive】　　極性表現数", self.sentiment)
            self._Print()
            s = pd.Series([self.buzz_id, self.date, self.buzz_name, self.buzz_full_text, "positive", self.sentiment["positive"], self.sentiment["negative"], self.followers, self.link], index = self.columns)
            self.posi_pd = self.posi_pd.append(s, ignore_index=True)
        elif self.sentiment["negative"]/total >= 0.7:
            print("==========")
            print(self.buzz_full_text)
            print()
            print("【判定:fire!!!】　　極性表現数", self.sentiment)
            print("ネガ表現の割合{:.3g}".format(self.sentiment["negative"]/total))
            self._Print()
            s = pd.Series([self.buzz_id, self.date, self.buzz_name, self.buzz_full_text, "fire", self.sentiment["positive"], self.sentiment["negative"], self.followers, self.link], index = self.columns)
            self.fire_pd = self.fire_pd.append(s, ignore_index=True)
        else:
            print("==========")
            print(self.buzz_full_text)
            print()
            print("【判定:negative】　　極性表現数", self.sentiment)
            self._Print()
            s = pd.Series([self.buzz_id, self.date, self.buzz_name, self.buzz_full_text, "negative", self.sentiment["positive"], self.sentiment["negative"], self.followers, self.link], index = self.columns)
            self.nega_pd = self.nega_pd.append(s, ignore_index=True)
        print()


### ツイートデータセット取得　実行

In [22]:
# 指定日のツイートを取得（API制限のため取得できるのは約一週間前のものまで）
day = '2020-1-16'
# リクエスト制限対応：True:リクエスト上限に達したら15分待機ののちツイート取得続行/ False:待機せずcsv取得
reload = True

#除外ワード
exclud_words = "配信スタート ＃キャンペーン　リツイートキャンペーン WWWWWWWWW wwwwwwwwww"

#その他設定可能パラメータ
#リプライをprint（print_rep = True/Fals), 最低RT数(RT_count = 5000)

GT = Get_Twitter(day, reload, exclud_words)
GT.main()

このスイマーバという商品が存在してくれたことに夫婦揃って心から感謝している。自由に手足を動かしまくれることを心から楽しんでいる。１日の中で明らかに最もイキイキとする時間。装着したとたんに大興奮する。ありがとうスイマーバ（のぼせさせないためにも目は絶対離してはダメ）。 

【判定:positive】　　極性表現数 {'positive': 128, 'negative': 68}
name： 7_color_world ／フォロワー数： 8529
date： 2020-01-15 07:47 ／ツイートID： 1217216822463746049
RT数： 5926 ／favorite数： 30745
リプライ数： 9 ／RTコメント数(上限１００）： 100

現役保育士だが、
つるの氏はともかく
『私は◯人育児してきたからプロフェッショナルよ！』と豪語できる人はむしろ保育現場向きではない。

我が子の成功論は必ずしも他の子には通用しない。むしろ邪魔になることもある。大事なのは目の前のこどもに真摯に向き合い続けること。
→

【判定:positive】　　極性表現数 {'positive': 169, 'negative': 125}
name： jiyuunaokan ／フォロワー数： 9597
date： 2020-01-15 07:42 ／ツイートID： 1217215495729844224
RT数： 5028 ／favorite数： 17391
リプライ数： 16 ／RTコメント数(上限１００）： 100

ネット・ゲーム依存症より遥かに深刻なのが大人の会社勤め依存症。週5〜7日，一回8時間以上(14時間以上という例も)も時間を費やしてしまい、生活に支障が出ている例も少なくないことから、
法整備を含め、政府や厚生労働省などの動きが加速することを期待する。

【判定:negative】　　極性表現数 {'positive': 101, 'negative': 181}
name： kzmakino ／フォロワー数： 4159
date： 2020-01-15 06:40 ／ツイートID： 1217199885881004032
RT数： 16044 ／favorite数： 24711
リプライ数： 49 ／RTコメント数(上限１００）： 80

AAAが僕

,Id,Date,Name,Full_text,Judge,Posi_score,Nega_score,Followers,link
0,1217216822463746049,2020-01-15 07:47,7_color_world,このスイマーバという商品が存在してくれたことに夫婦揃って心から感謝している。自由に手足を動か...,positive,128,68,8529,https://t.co/6r8g80JmlE
1,1217215495729844224,2020-01-15 07:42,jiyuunaokan,現役保育士だが、\nつるの氏はともかく\n『私は◯人育児してきたからプロフェッショナルよ！』...,positive,169,125,9597,None
2,1217198528297963520,2020-01-15 06:35,WJF_SHIROSE,AAAが僕にくれたもの。\n\n「Attack All Around」\n,positive,141,37,118153,https://t.co/pGSiZ3FNN8
3,1217186712759128064,2020-01-15 05:48,danmenzukan,本日1月15日は #いちごの日 🍓💕\nという訳で念願の #苺の家系図、完成したので公開しま...,positive,80,33,330,https://t.co/CUT2RuGX0D
4,1217165290003263489,2020-01-15 04:23,ApexTimes,今回のアップデートからジブラルタルがドーム内で味方を蘇生するとモーションが変化し、蘇生が速く...,positive,29,17,47914,https://t.co/FXPQsOk2gV



↓↓↓negativeサンプル↓↓↓


,Id,Date,Name,Full_text,Judge,Posi_score,Nega_score,Followers,link
0,1217199885881004032,2020-01-15 06:40,kzmakino,ネット・ゲーム依存症より遥かに深刻なのが大人の会社勤め依存症。週5〜7日，一回8時間以上(1...,negative,101,181,4159,None
1,1217100137668870144,2020-01-15 00:04,AAA_staff,【AAAから大切なお知らせ】\nAAAからファンクラブ会員の皆さまに大切なお知らせがございま...,negative,60,63,625344,https://t.co/QOtrdorAnH
2,1217069457874407431,2020-01-14 22:02,Ryang_Fang4,WANIMAのオタクくん構文、本人達が結構嫌がってるらしく、Twitterに生息するオタクの...,negative,76,119,1197,None



↓↓↓fire_tweetサンプル↓↓↓


,Id,Date,Name,Full_text,Judge,Posi_score,Nega_score,Followers,link




csvへの書き出しが完了しました。新規データ数18、全データ数：514
サンプルが0件の場合は、15分後に再度実行すると取得できる場合があります。
fire_tweetは出現率が非常に低いです。


## ２）データセットの前処理
　・正規表現、ストップワード除去など
 
 #### 参考サイト
Pythonで全角・半角記号をまとめて消し去る<br>　http://prpr.hatenablog.jp/entry/2016/11/23/Python%E3%81%A7%E5%85%A8%E8%A7%92%E3%83%BB%E5%8D%8A%E8%A7%92%E8%A8%98%E5%8F%B7%E3%82%92%E3%81%BE%E3%81%A8%E3%82%81%E3%81%A6%E6%B6%88%E3%81%97%E5%8E%BB%E3%82%8B

In [11]:
#必要なツールをインストール(初回のみ実行)
! pip install gensim
! pip install natto-py
! pip install emoji

  Created wheel for natto-py: filename=natto_py-0.9.2-cp36-none-any.whl size=50571 sha256=65c64e8f0f386ee150bb2ab7d8bbef980b44df48abc5c100e20583126ac1b3d0
  Stored in directory: /root/.cache/pip/wheels/ce/51/dd/67f87608b124a23eecf5c1fc3557cc0b7ffdeae33fe6ee89df
Successfully built natto-py


In [23]:
#ツイートデータを学習用に整形
#from natto import MeCab
import MeCab
import re
import pandas as pd
import pprint
import emoji
import neologdn
import urllib.request
import unicodedata
import string

class For_Model():
    
    def __init__(self, data, columns, out_file, mode, text, similar = None):
        self.mecab = MeCab.Tagger("-Owakati")
        self.data = data
        self.columns = columns
        self.out_file = out_file
        self.mode = mode
        self.text = text
        self.similar = str(similar)

    #データを読み込む
    def Load_tweets(self):        
        df = pd.read_csv(self.data, usecols = self.columns)
        print("読み込んだツイート", df.shape[0])
        
        #３０w以下のtweet行を削除
        index = []
        for i in range(len(df)):
            line = df.iloc[i]
            text = str(line["Full_text"])
            text = re.sub('https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
            text = re.sub('http?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
            line["Full_text"] = text
            if len(text) < 30:
                index.append(i)
        df_tweet = df.drop(df.index[index])
        df_tweet = df_tweet.reset_index(drop=True)
        print("30w以下削除後のツイート数", df_tweet.shape[0])
        display(df_tweet.head())
        
        #判定用テキストをリストの最後に追加
        tweets = []
        for i in df_tweet[self.text]:
            tweets.append(i)
        if self.similar == None:
            pass
        else:
            tweets.append(self.similar)
        return df_tweet, tweets

    def Stop_Words(self):
        # ストップワードをダウンロード
        url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
        urllib.request.urlretrieve(url, './output/stop_word.txt')

        with open('./output/stop_word.txt', 'r', encoding='utf-8') as file:
            stopwords = [word.replace('\n', '') for word in file.readlines()]

        #追加ストップワードを設定（助詞や意味のない平仮名１文字）
        add_words = ['あ','い','う','え','お','か','き','く','け','こ','さ','し','す','せ','そ','た','ち','つ','て','と',
                     'な','に','ぬ','ね','の','は','ひ','ふ','へ','ほ','ま','み','む','め','も','や','ゆ','よ',
                     'ら','り','る','れ','ろ','わ','を','ん','が','ぎ','ぐ','げ','ご','ざ','じ','ず','ぜ','ぞ',
                     'だ','ぢ','づ','で','ど','ば','び','ぶ','べ','ぼ','ぱ','ぴ','ぷ','ぺ','ぽ',
                     'くん','です','ます','ました','そして','でも','だから','だが','くらい','その','それ','かも',
                     'あれ','あの','あっ','そんな','この','これ','とか','とも','する','という','ござい',
                     'ので','なんて','たら', 'られ','たい','さて','てる','ください','なる','けど','でし',
                     'じゃん','だっ','なっ','でしょ', 'ある','って','こんな','ねえ'
                    ]
        stopwords = stopwords + add_words
        return stopwords

    def Tokenizer(self, text, stopwords):

        words = []
        text = self.mecab.parse(text)
        text = text.split(' ')
        for j in range(len(text)):
            if text[j] not in stopwords:
                words.append(text[j])
        return words

    def remove_emoji(self, text):
        return ''.join(c for c in text if c not in emoji.UNICODE_EMOJI)

    #記号削除
    def format_text(self, text):
        text = unicodedata.normalize("NFKC", text)  # 全角記号を半角へ置換
        # 記号を消し去るための魔法のテーブル作成
        table = str.maketrans("", "", string.punctuation  + "「」、。・*`+-|?#!()\[]<>=~/")
        text = text.translate(table)
        return text

    def main(self):
        tweets_num = 0
        stopwords = self.Stop_Words()
        df_tweet, tweets = self.Load_tweets()
        #ツイートを分かち書きしてcsvに出力(モード'a'はデータ追加、モード'w'は新規作成)
        with open('./output/' + self.out_file, self.mode) as f:
            for i in tweets:
                tweets_num += 1
                i = neologdn.normalize(i)
                i = re.sub('\n', "", i)
                i = re.sub(r'[!-~]', "", i)#半角記号,数字,英字
                i = re.sub(r'[︰-＠]', "", i)#全角記号
                i = self.format_text(i)#記号削除
                i = re.sub(r'[【】●ㅅ●Ф☆✩︎♡→←▼①②③④⑤『』ω《》∠∇∩♪∀◞ཀCщ≧≦ ́◤◢■◆★※↑↓〇◯○◎⇒▽◉Θ♫♬〃“”◇ᄉ⊂⊃д°]', "", i)
                #i = re.sub(r'[!-~、。‥…？！〜「」｢｣:：“”【】※♪♩♫♬『』→↓↑《》〈〉[]≧∇≦・゜・●ㅅ●´Д´°ω°•ω•★＊☆♡（）✔Θ∀´∀｀˘ω˘‼бωб￣▽￣◉→←▼①②③④⑤]', "", i)
                i = self.remove_emoji(i)
                i = self.Tokenizer(i, stopwords)
                i = ' '.join(i) #リストを文字列に変換
                i = str(i)
                f.write(i)

        print('CSV出力完了：'+ self.out_file)
        with open('./output/' + self.out_file) as f:
             wakati = f.read()

        print("学習用データに書き込んだツイート数（判定用ツイート含む）：", tweets_num)
        print()
        print("分かち書きサンプル\n", wakati[:500])
        return df_tweet


### 前処理の実行

In [24]:
#パラメータの設定

#取得したデータのパス
data = './output/buzz_tweet.csv'
#取得したい列名
columns = ["Followers", "Full_text","Posi_score", "Nega_score","Judge"]
#出力ファイル名
out_file = "train_buzz.txt"
#学習データの保存モード　'a'：追加／'w'：上書き
mode = 'w'
#ツイートテキストの列を指定
text = "Full_text"
#判定させたいツイート予定文書（類似度確認のため、データセット内にあるツイート文を使用）
similar = "電車内で咳やくしゃみをしている人、マスクをしないのが信じられません。ウイルスをばら撒いている自覚がないのだろうか。頼むからマスクしてくれ。"
FM = For_Model(data, columns, out_file, mode, text, similar)
df_tweet = FM.main()

読み込んだツイート (514, 5)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


30w以下削除後のツイート数 (459, 5)


,Followers,Full_text,Judge,Nega_score,Posi_score
0,1564.0,絶対断らないと評判の病児保育室、助成金下りず2億円の赤字を出し閉鎖\n\n全国に2886カ所...,positive,148.0,168.0
1,2217.0,全国の皆さんへ\nどうか皆様のお力を貸してください。\n\n１日も早く娘を助けたいです。\n...,positive,50.0,129.0
2,3756.0,o0( 歴史上、さんざん他国の料理を魔改造してきた我が国が「寿司ポリス」などどは片腹痛い！あ...,positive,85.0,137.0
3,10999.0,これすんごい。GABAのフォースリープ、ツイッターで人気だったから半信半疑で夜に3粒食べてみ...,positive,78.0,97.0
4,56685.0,TVアニメ「Ｄｒ．ＳＴＯＮＥ」第2期制作が決定いたしました！第1期をご視聴・応援くださってい...,positive,20.0,156.0


CSV出力完了：train_buzz.txt
学習用データに書き込んだツイート数（判定用ツイート含む）： 460

分かち書きサンプル
 絶対 断ら ない 評判 病児保育 助成金 下り 赤字 出し 閉鎖 全国 病児保育 赤字 運営 おり 東海 キッズケア 助成金 求め 署名 集め 助成金 下りる あり ませ 社会保障 税金 使わ ませ 
全国 皆さん どうか 皆様 お力 貸し 早く 娘 助け 家族 揃っ 笑顔 クリスマス 新年 迎え 娘 目撃 情報 娘 繋がる 些細なこと 連絡 連絡先 大月 警察 暑 電話 もしくは 最寄り 警察 署 拡散希望 小倉 美咲 
歴史 さんざん 他国 料理 魔 改造 我が国 寿司 ポリス どど 片腹痛い あらゆる 文化 文化 独自 解釈 いい イノベーション 生む 思っ しかし ヘルシンキ 〈 クリスマス トッピング • バナナ 巻き寿司 〉 寛容 試さ いる 
すん ごい フォースリープツイッター 人気 半信半疑 夜 粒 食べ 昨日 めちゃ ぐっすり 眠れ 例える なら 旅行 気持ち良く 遊び 疲れ 夜 睡眠 クオリティ しかも 味 まろやか ミルク チョコ 好み すぎる 目安 一日 粒 摂 れる 
アニメ 期 制作 決定 いたし 期 視聴 応援 くださっ いる 皆様 本当に ありがとう 


## ３）予測モデルを生成
　・データセットをDoc２vecで学習<br>

#### 参考サイト
fastTextとDoc2Vecのモデルを作成してニュース記事の多クラス分類の精度を比較する<br> https://qiita.com/kazuki_hayakawa/items/ca5d4735b9514895e197<br>

In [26]:
#Doc2Vecモデルの学習

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

f = open('./output/train_buzz.txt','r')#空白で単語を区切り、改行で文書を区切っているテキストデータ

#１文書ずつ、単語に分割してリストに入れていく[([単語1,単語2,単語3],文書id),...]こんなイメージ
#words：文書に含まれる単語のリスト（単語の重複あり）
# tags：文書の識別子（リストで指定．1つの文書に複数のタグを付与できる）
#fにテキスト データをいれる
trainings = [TaggedDocument(words = data.split(),tags = [i]) for i,data in enumerate(f)]
#print(type(trainings))
print("Doc２vec文書ベクトル用モデルに学習させたツイート数",len(trainings))
# print(trainings[:20])

#文書ベクトル用ツイートテキストの学習(パラメータ：dm=1:dmpv それ以外：DBow)
model = Doc2Vec(
    documents= trainings,
    dm = 1,
    vector_size=300,
    window=5,
    alpha = 0.05,
    min_count=1,
    sample = 0,
    workers=4, 
    epochs = 1000
)

#出力用ディレクトリ作成（存在しない場合のみ）
def Make_Dir():
    new_dir_path = 'model'
    try:
        os.makedirs(new_dir_path)
    except FileExistsError:
        pass

# モデルのセーブ
Make_Dir()
model.save("./model/doc2vec.model")

# モデルのロード(モデルが用意してあれば、ここからで良い)
m = Doc2Vec.load('./model/doc2vec.model')

Doc２vec文書ベクトル用モデルに学習させたツイート数 460


## ４）ツイート予定文章のネガポジ予測を返す
　・データセットから、入力しておいたツイート予定文書と似ている文書を探す<br>
・ネガポジスコア付きで、類似ツイート上位１０個を返す<br>
#### 結果：成功。入力文書と同じツイート文が類似度１位に。ネガポジもデータズレなく表示できた

In [27]:
import numpy as np
import math

#類似判定と類似している上位10件の文書を出力

top10 = m.docvecs.most_similar(len(trainings) - 1)

print("=========== 判定したいツイート ===========\n")
print(similar)

print()
print("======= 類似度上位１０（全{}ツイート中） =======".format(len(trainings)))
print("※nega/posiスコアはフォロワー数のlogで割っています。フォロワー数はばらつきが大きいので対数変換しています")
print()
for i in range(len(top10)):
    score = top10[i]
    index = int(score[0])
    similar_score = score[1]
    tweet = df_tweet["Full_text"]
    judge = df_tweet["Judge"]
    posi_score = df_tweet["Posi_score"]
    nega_score = df_tweet["Nega_score"]
    followers = df_tweet["Followers"]
    print("…………　類似ツイート{}位：類似度 {}　…………".format((i+1), math.floor(similar_score*1000)/1000))
    print()
    print(tweet[index])
    print()
    print("【極性】：", judge[index])
    print("posi_score：",math.floor((posi_score[index]/np.log(followers[index]))*100)/100, "／", "nega_score：", math.floor((nega_score[index]/np.log(followers[index]))*100)/100, "／followers：", math.floor(followers[index]))

    print()


=========== 判定したいツイート ===========

電車内で咳やくしゃみをしている人、マスクをしないのが信じられません。ウイルスをばら撒いている自覚がないのだろうか。頼むからマスクしてくれ。

======= 類似度上位１０（全460ツイート中） =======
※nega/posiスコアはフォロワー数のlogで割っています。フォロワー数はばらつきが大きいので対数変換しています

…………　類似ツイート1位：類似度 0.726　…………

AAAが僕にくれたもの。

「Attack All Around」


【極性】： positive
posi_score： 12.07 ／ nega_score： 3.16 ／followers： 118153

…………　類似ツイート2位：類似度 0.698　…………

そんな、まさか❗️　あいつは…

@MorbiusMovieJP 

【極性】： positive
posi_score： 4.82 ／ nega_score： 3.82 ／followers： 164741

…………　類似ツイート3位：類似度 0.679　…………

天皇ですら人間宣言したのにただの客が神様なわけねぇだろボケナス大逆罪共が

【極性】： positive
posi_score： 11.29 ／ nega_score： 6.83 ／followers： 835

…………　類似ツイート4位：類似度 0.667　…………

上手すぎだろwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww 

【極性】： positive
posi_score： 11.45 ／ nega_score： 3.2 ／followers： 378

…………　類似ツイート5位：類似度 0.657　…………

マスクせずに咳してる人見ると、
「感染者だ！撃ち殺せ！」
「ですが！アイツはまだ人間じゃないですか！」
「馬鹿野郎！ここで殺らなきゃ、俺達の大事な奴まで感染者にされる可能性だってあるんだ！迷うな！引き金を引け！」
「畜生！感染者め！」
みたいな気持ちになる。マスクしやがり下さい。

【極性】： negative
posi_sc

# その他試みたこと
断念、または精度が全く良くない。覚書として記録

## １）文章ベクトルを特徴量としたネガポジ予測モデル
　・文章ベクトルとフォロワー数を特徴量X、ネガポジスコアを目的変数yとしたデータを学習<br>
　・文章ベクトルはDoc２vecとTf-idfの２種を作成<br>
　・ツイート予定文書を入力してネガポジスコアを予測する<br>
　・試した予測モデル<br>
　・MultiOutputRegressor、SVRのrbf と　SVRの線形、lightgbm、ランダムフォレスト<br>
#### 結果：精度が低すぎて断念<br>

### Doc2vecで文章ベクトル取得

In [27]:
#Doc2vecでベクトル化
from natto import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer

df_buzz = pd.read_csv('./output/buzz_tweet.csv',
                      usecols = ["Full_text", "Posi_score", "Nega_score", "Followers"])
#.to_csv('./output/for_training.csv', mode = "a", index = False, header = None)
#pd.read_csv('./output/fire_buzz_tweet.csv', usecols = ["Full_text", "Judge", "Sentiment"]).to_csv('./output/for_training.csv', mode = "a", index = False, header = None)
print("ベクトル化するセンチメントスコア付きデータ数：", len(df_buzz))
display(df_buzz.head())

#doc2vecでベクトル化
for_training = df_buzz['Full_text']
#print(for_training)
vector_tweet = []
for i in for_training:
    i = m.infer_vector(i)
    vector_tweet.append(i)

df_vector = pd.DataFrame(data = vector_tweet)

# print("Doc2vecベクトル")
# display(df_vector.head())

ベクトル化するセンチメントスコア付きデータ数： 374


,Followers,Full_text,Nega_score,Posi_score
0,6800.0,君、すごい食い方やな\n https://t.co/yRTGvd43wT,31.0,75.0
1,1564.0,絶対断らないと評判の病児保育室、助成金下りず2億円の赤字を出し閉鎖\n\n全国に2886カ所...,148.0,168.0
2,2217.0,全国の皆さんへ\nどうか皆様のお力を貸してください。\n\n１日も早く娘を助けたいです。\n...,50.0,129.0
3,3756.0,o0( 歴史上、さんざん他国の料理を魔改造してきた我が国が「寿司ポリス」などどは片腹痛い！あ...,85.0,137.0
4,7363.0,百合関係図です。 https://t.co/hZxIYOsSag,58.0,82.0


### Tf-idfでベクトル取得
#### 参考サイト

機械学習_サポートベクターマシーン_pythonで実装<br>
https://dev.classmethod.jp/machine-learning/2017ad_20171214_svm_python/<br>
Tf-idfベクトルってなんだ？<br> https://qiita.com/MasatoTsutsumi/items/5b0a140b1ecbdd0396e1

In [28]:
# 2-1.tf-idf計算
from sklearn.feature_extraction.text import TfidfVectorizer

def Stop_Words():
    # ストップワードをダウンロード
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    urllib.request.urlretrieve(url, './output/stop_word.txt')

    with open('./output/stop_word.txt', 'r', encoding='utf-8') as file:
        stopwords = [word.replace('\n', '') for word in file.readlines()]

    #追加ストップワードを設定（助詞や意味のない平仮名１文字）
    add_words = ['あ','い','う','え','お','か','き','く','け','こ','さ','し','す','せ','そ','た','ち','つ','て','と',
                 'な','に','ぬ','ね','の','は','ひ','ふ','へ','ほ','ま','み','む','め','も','や','ゆ','よ',
                 'ら','り','る','れ','ろ','わ','を','ん','が','ぎ','ぐ','げ','ご','ざ','じ','ず','ぜ','ぞ',
                 'だ','ぢ','づ','で','ど','ば','び','ぶ','べ','ぼ','ぱ','ぴ','ぷ','ぺ','ぽ',
                 'くん','です','ます','ました','そして','でも','だから','だが','くらい','その','それ','かも',
                 'あれ','あの','あっ','そんな','この','これ','とか','とも','する','という','ござい',
                 'ので','なんて','たら', 'られ','たい','さて','てる','ください','なる','けど','でし',
                 'じゃん','だっ','なっ','でしょ', 'ある','って','こんな','ねえ'
                ]
    stopwords = stopwords + add_words
    return stopwords

stopwords = Stop_Words()
tfidfv = TfidfVectorizer(lowercase=True, stop_words=stopwords) # stop words処理
 
tfv_vector_fit = tfidfv.fit(for_training)
tfv_vector = tfv_vector_fit.transform(for_training)
print(tfv_vector.shape) 

# 2-2.次元削減(「lsa」を使って次元削減を行う)
from sklearn.decomposition import TruncatedSVD

# 2-2-1.パラメータの調整
list_n_comp = [5,10,50,100,500,1000] # 特徴量を何個に削減するか、というパラメータです。できるだけ情報量を欠損しないで、かつ次元数は少なくしたいですね。
for i in list_n_comp:
    lsa = TruncatedSVD(n_components=i,n_iter=5, random_state = 0)
    lsa.fit(tfv_vector) 
    tfv_vector_lsa = lsa.transform(tfv_vector)
    print('次元削減後の特徴量が{0}の時の説明できる分散の割合合計は{1}です'.format(i,round((sum(lsa.explained_variance_ratio_)),2)))

# 2-2-2.次元削減した状態のデータを作成
# 上記で確認した「n_components」に指定した上で、次元削減（特徴抽出）を行う
lsa = TruncatedSVD(n_components=1000, n_iter=5, random_state = 0) # 今回は次元数を1000に指定
lsa.fit(tfv_vector)
X_tf = lsa.transform(tfv_vector)
print()
print("次元削減後Tf-idf\n", X_tf.shape)
# print(X_tf)

(374, 2402)
次元削減後の特徴量が5の時の説明できる分散の割合合計は0.04です
次元削減後の特徴量が10の時の説明できる分散の割合合計は0.05です
次元削減後の特徴量が50の時の説明できる分散の割合合計は0.17です
次元削減後の特徴量が100の時の説明できる分散の割合合計は0.3です
次元削減後の特徴量が500の時の説明できる分散の割合合計は1.0です
次元削減後の特徴量が1000の時の説明できる分散の割合合計は1.0です

次元削減後Tf-idf
 (374, 374)


In [29]:
#X、yデータを作成

#Doc2vecのベクトルデータ
print("欠損値削除前データ", df_buzz.shape)
print()

#文書ベクトルを含んだdf
df_buzz_vec = pd.concat([df_buzz, df_vector], axis=1)
df_buzz_vec = df_buzz_vec.dropna(subset = ["Followers"])#欠損値行削除
df_buzz_vec = df_buzz_vec.drop([ "Full_text", "Nega_score", "Posi_score"], axis=1)
X = df_buzz_vec.values
print("Doc2vecベクトル")
print("X.shape", X.shape)
display(df_buzz_vec.head())

#tf-idfのベクトルデータ
tf_df = pd.DataFrame(data = X_tf)
tf_df = pd.concat([df_buzz, tf_df], axis=1)
tf_df = tf_df.dropna(subset = ["Followers"])#欠損値行削除
tf_df = tf_df.drop([ "Full_text", "Nega_score", "Posi_score"], axis=1)
X_tf_idf = tf_df.values
print("Tf-idfベクトル")
print("X_tf_idf.shape", tf_df.shape)
display(tf_df.head())

#yデータ作成
df_buzz = df_buzz.dropna(subset = ["Followers"])#y用に"Followers"の欠損行削除
y = df_buzz.loc[:,['Posi_score', 'Nega_score']]#できればDateも特徴量に入れたい
y_p = df_buzz['Posi_score']
y_n = df_buzz['Nega_score']
y = y.values
print()
print("y.shape", y.shape)
y_p = y_p.values
y_n = y_n.values

欠損値削除前データ (374, 4)

Doc2vecベクトル
X.shape (373, 301)


,Followers,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,6800.0,-0.002347,0.004516,-0.002467,0.017913,0.006379,0.001529,-0.013634,-0.001045,0.017018,...,-0.014283,-0.010716,-0.018207,0.013203,0.021252,-0.011075,-0.009633,-0.008679,-0.001196,-0.003732
1,1564.0,0.001994,0.027724,-0.005945,0.029363,0.015228,0.030413,-0.002512,0.003670,0.026079,...,-0.032680,-0.021952,-0.063932,0.006111,0.054873,-0.017913,-0.025832,-0.019413,0.003702,-0.002683
2,2217.0,-0.008996,0.021177,-0.013813,0.025786,0.017142,0.016225,-0.025777,0.001117,0.025575,...,0.001572,0.009682,-0.025858,0.006571,0.017764,0.009201,-0.001735,0.008258,-0.002077,0.001832
3,3756.0,-0.010936,-0.010247,-0.003010,0.031322,0.007310,0.002045,-0.033298,-0.004619,0.028420,...,-0.015579,-0.020676,-0.011868,0.030162,0.037181,-0.023565,-0.011537,-0.002477,-0.002884,-0.002214
4,7363.0,-0.000134,0.011890,-0.008067,0.019329,0.007480,0.015009,-0.012856,0.001064,0.013483,...,-0.003167,-0.000344,-0.024332,0.003653,0.017458,0.000162,-0.002972,0.002077,0.002838,-0.002779


Tf-idfベクトル
X_tf_idf.shape (373, 375)


,Followers,0,1,2,3,4,5,6,7,8,...,364,365,366,367,368,369,370,371,372,373
0,6800.0,0.308188,-0.000429,-0.000171,-0.002650,-1.302487e-16,-0.001072,-0.003650,-0.008917,-0.015199,...,-0.000030,-0.000064,-0.000972,-0.003923,0.000312,-0.000123,0.000327,1.482354e-17,-0.000037,-0.000026
1,1564.0,0.184497,-0.000264,-0.000108,-0.001684,7.847122e-17,-0.000692,-0.002466,-0.006032,-0.010359,...,-0.000015,-0.000032,-0.000497,-0.002015,0.000163,-0.000065,0.000176,-9.693617e-17,-0.000020,-0.000014
2,2217.0,0.115796,-0.000171,-0.000074,-0.001162,-1.803444e-16,-0.000493,-0.001979,-0.004864,-0.008556,...,-0.000008,-0.000018,-0.000276,-0.001123,0.000091,-0.000037,0.000100,1.058502e-16,-0.000011,-0.000008
3,3756.0,0.118023,-0.000170,-0.000070,-0.001099,-7.856138e-17,-0.000454,-0.001647,-0.004030,-0.006941,...,-0.000009,-0.000020,-0.000303,-0.001232,0.000100,-0.000040,0.000109,-4.674257e-17,-0.000012,-0.000009
4,7363.0,0.308188,-0.000429,-0.000171,-0.002650,-1.911629e-16,-0.001072,-0.003650,-0.008917,-0.015199,...,-0.000030,-0.000064,-0.000972,-0.003923,0.000312,-0.000123,0.000327,1.141115e-16,-0.000037,-0.000026



y.shape (373, 2)


### MultiOutputRegressorで複数の回帰¶
#### 結果：D2vベクトルよりTf-idfがややマシ

In [30]:
#MultiOutputRegressorで複数の回帰

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

#D2vベクトル
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=0)

#X, y = make_regression(n_samples=10, n_targets=3, random_state=1)
MOR = MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X_train, y_train)
y_pred = MOR.predict(X_test)
score = MOR.score(X_test, y_test)

print("正解\n", y_test)
print()
print(y_pred)
print("R ^ 2_score(1に近いほど良い）：", score)
print()

#X_tf tf-idfベクトルを使った予測
X_train, X_test, y_train, y_test = train_test_split(
    X_tf_idf, y, train_size=0.7, test_size=0.3, random_state=0)

MOR = MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X_train, y_train)
y_pred = MOR.predict(X_test)
score = MOR.score(X_test, y_test)
print(y_pred)
print("R ^ 2_score(1に近いほど良い）：", score)

正解
 [[ 141.   39.]
 [ 123.  139.]
 [  77.   32.]
 [  40.   63.]
 [  93.  125.]
 [  91.   89.]
 [  99.   24.]
 [ 141.  131.]
 [  91.   29.]
 [  94.   62.]
 [  46.   51.]
 [  48.   32.]
 [  49.   58.]
 [ 116.  142.]
 [  76.   46.]
 [  88.   24.]
 [ 124.  102.]
 [  54.   20.]
 [  75.   29.]
 [ 122.   48.]
 [ 266.   76.]
 [ 118.  224.]
 [  25.   27.]
 [ 104.   84.]
 [ 118.  227.]
 [ 128.    8.]
 [ 125.   43.]
 [  68.   42.]
 [  60.   28.]
 [ 168.  148.]
 [  40.   45.]
 [ 103.   56.]
 [  71.   58.]
 [  65.   43.]
 [  61.   46.]
 [  88.   24.]
 [ 156.   33.]
 [  59.   54.]
 [  18.   46.]
 [ 102.   81.]
 [ 114.   43.]
 [  92.   69.]
 [ 160.  144.]
 [ 133.   28.]
 [  55.   59.]
 [ 117.   26.]
 [  63.   29.]
 [ 205.   45.]
 [  97.   78.]
 [  40.   80.]
 [ 100.   27.]
 [ 126.  124.]
 [ 121.   16.]
 [  31.   37.]
 [ 116.   83.]
 [  28.   16.]
 [  62.   36.]
 [  69.   20.]
 [  37.   49.]
 [ 179.   41.]
 [ 140.  120.]
 [ 121.   78.]
 [  79.   21.]
 [  97.   47.]
 [  38.   42.]
 [  52.   15.]
 [  85

### SVRのrbf と　SVRの線形で予測
#### 結果：予測値が全くダメ

In [31]:
#ポジ、ネガ別々で予測する
#SVRのrbf と　SVRの線形

import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from math import sqrt

#D2vベクトル(ポジのみ)
X_train, X_test, y_p_train, y_p_test = train_test_split(
    X, y_p, train_size=0.7, test_size=0.3, random_state=0)

svr_rbf = SVR(kernel='rbf', C=1, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1)
y_rbf = svr_rbf.fit(X_train, y_p_train)
y_lin = svr_lin.fit(X_train, y_p_train)

pred_rbf = svr_rbf.predict(X_test)
pred_lin = svr_lin.predict(X_test)

#精度

# 相関係数計算
rbf_corr = np.corrcoef(y_p_test, pred_rbf)[0, 1]
lin_corr = np.corrcoef(y_p_test, pred_lin)[0, 1]

# RMSEを計算（０に近いほど良い）
rbf_rmse = sqrt(mean_squared_error(pred_rbf, y_p_test))
lin_rmse = sqrt(mean_squared_error(pred_lin, y_p_test))

print("RBF: RMSE（０に近いほど良い） {} ".format(rbf_rmse))
print("Linear: RMSE（０に近いほど良い） {}" .format(lin_rmse))
print()
print("正解", y_p_test)
print("rbf推定", pred_rbf)
print("lin推定", pred_lin)


RBF: RMSE（０に近いほど良い） 46.977801729985174 
Linear: RMSE（０に近いほど良い） 5332.519130224009

正解 [ 141.  123.   77.   40.   93.   91.   99.  141.   91.   94.   46.   48.
   49.  116.   76.   88.  124.   54.   75.  122.  266.  118.   25.  104.
  118.  128.  125.   68.   60.  168.   40.  103.   71.   65.   61.   88.
  156.   59.   18.  102.  114.   92.  160.  133.   55.  117.   63.  205.
   97.   40.  100.  126.  121.   31.  116.   28.   62.   69.   37.  179.
  140.  121.   79.   97.   38.   52.   85.   81.  102.   77.  122.  156.
   90.   44.  102.   72.   92.  107.  171.  186.  190.  119.   39.  214.
  115.  149.   57.  122.   26.   67.  168.  146.   21.  169.   38.  118.
   44.   79.   58.   95.   41.   89.   64.   77.   48.   94.   97.   83.
   70.   51.  210.   87.]
rbf推定 [ 92.61958406  91.72157239  91.72157239  91.72157239  91.7212691
  91.72157244  91.72157239  91.72157239  91.72157239  91.72157239
  91.72157239  91.72157239  91.72157239  91.72157239  91.71672799
  91.7231354   91.72157239  9

### lightgbm
#### 参考サイト

Mercari Price Challenge -機械学習を使ったメルカリの価格予測 Ridge回帰 LightGBM

http://rautaku.hatenablog.com/entry/2017/12/22/195649

#### 結果：RMSEが0には程遠い

In [32]:
#必要なツールをインストール(初回のみ実行)
! pip install lightgbm

In [33]:
#LightGBM を使った回帰予測(D2Vベクトル)

import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

def main():
    #D2vベクトル(ポジのみ)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_p, train_size=0.7, test_size=0.3, random_state=0)

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # LightGBM のハイパーパラメータ
    lgbm_params = {
        # 回帰問題
        'objective': 'regression',
        # RMSE (平均二乗誤差平方根) の最小化を目指す
        'metric': 'rmse',
    }

    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train, 
                      valid_sets=lgb_eval, num_boost_round=8000, 
                      early_stopping_rounds=5000, verbose_eval=500)

    # テストデータを予測する
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # RMSE を計算する
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("RMSE（０に近いほど良い）", rmse)

if __name__ == '__main__':
    main()

Training until validation scores don't improve for 5000 rounds
[500]	valid_0's rmse: 55.8736
[1000]	valid_0's rmse: 56.1001
[1500]	valid_0's rmse: 56.1358
[2000]	valid_0's rmse: 56.1429
[2500]	valid_0's rmse: 56.1444
[3000]	valid_0's rmse: 56.1446
[3500]	valid_0's rmse: 56.1447
[4000]	valid_0's rmse: 56.1447
[4500]	valid_0's rmse: 56.1447
[5000]	valid_0's rmse: 56.1447
Early stopping, best iteration is:
[5]	valid_0's rmse: 46.8401
RMSE（０に近いほど良い） 46.8400537327


In [34]:
#LightGBM を使った回帰予測（Tfーidfベクトル）

def main():

    #X_tf tf-idfベクトルを使った予測(ポジのみ)
    X_train, X_test, y_train, y_test = train_test_split(
        X_tf_idf, y_p, train_size=0.7, test_size=0.3, random_state=0)

    # データセットを生成する
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # LightGBM のハイパーパラメータ
    lgbm_params = {
        # 回帰問題
        'objective': 'regression',
        # RMSE (平均二乗誤差平方根) の最小化を目指す
        'metric': 'rmse',
    }
    
    # 上記のパラメータでモデルを学習する
    model = lgb.train(lgbm_params, lgb_train, 
                      valid_sets=lgb_eval, num_boost_round=8000, 
                      early_stopping_rounds=5000, verbose_eval=500)
#     model = lgb.LGBMRegressor()
#     model.fit(X_train, y_train)

    # テストデータを予測する
    y_pred = model.predict(X_test)

    # RMSE を計算する
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("RMSE（０に近いほど良い）",rmse)


if __name__ == '__main__':
    main()

Training until validation scores don't improve for 5000 rounds
[500]	valid_0's rmse: 51.6926
[1000]	valid_0's rmse: 51.8494
[1500]	valid_0's rmse: 51.8677
[2000]	valid_0's rmse: 51.8695
[2500]	valid_0's rmse: 51.8698
[3000]	valid_0's rmse: 51.8698
[3500]	valid_0's rmse: 51.8698
[4000]	valid_0's rmse: 51.8698
[4500]	valid_0's rmse: 51.8698
[5000]	valid_0's rmse: 51.8698
Early stopping, best iteration is:
[1]	valid_0's rmse: 46.7848
RMSE（０に近いほど良い） 46.7848457582


### ランダムフォレスト

In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

#D2vベクトル
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=0)
# ランダムフォレスト回帰オブジェクト生成
rfr = RandomForestRegressor(n_estimators=100)
# 学習の実行
rfr.fit(X_train, y_train)
# テストデータで予測実行
predict_y = rfr.predict(X_test)
# R2決定係数で評価
r2_score = r2_score(y_test, predict_y)
print("R^2(1に近いほど良い）:", r2_score)


R^2(1に近いほど良い）: -0.1350692768


## ２）ツイッターAPI制限への挑戦（データセットの拡大）
　・古いツイートを大量取得できるパッケージを発見（通常は１週間程度しか遡れない）<br>
#### 結果：取得データから反応ツイートの取得を試みたができなかった<br>

### GetOldTweets3 0.0.11
古いツイートをトークン申請なしで大量取得できるパッケージ<br>
https://pypi.org/project/GetOldTweets3/